In [1]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
from datetime import date

# EXIT/IN

In [2]:
# exit_URL = 'https://exitin.com/calendar/'
# headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
# exit_response = requests.get(exit_URL, headers=headers)
# exit_webpage = BS(exit_response.content, 'lxml')

In [3]:
# exit_narrow = exit_webpage.find('div', attrs={'class':'calendar'})

# exit_artist_list = []
# exit_date_list = []
# exit_time_list = []
# exit_tickets_list = []

# exit_artist = exit_narrow.find_all('h1')[1:26:]
# for artist in exit_artist:
#     exit_artist_list.append(artist.text)
# exit_date = exit_narrow.find_all('div', attrs={'class':'name'})[1:200:8]
# for date in exit_date:
#     exit_date_list.append(date.text)
# exit_time = exit_narrow.find_all('div', attrs={'class':'name'})[4:200:8]
# for time in exit_time:
#     exit_time_list.append(time.text)
# exit_tickets = exit_narrow.find_all('div', attrs={'class':'name'})[6:200:8]
# for tickets in exit_tickets:
#     exit_tickets_list.append(tickets.text)
# exit_dict = {'event_date':exit_date_list,'event_name':exit_artist_list, 'show_time':exit_time_list, 'price':'Click For More Info', 'ticket_url':exit_tickets_list, 'Venue':'Exit/In'}

# exit_df = pd.DataFrame(exit_dict)
# exit_df

# MARATHON

In [4]:
MMW_URL = 'https://www.marathonmusicworks.com/tickets-2/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
mmw_response = requests.get(MMW_URL, headers=headers)
mmw_webpage = BS(mmw_response.content, 'lxml')

In [5]:
mmw_calendar = mmw_webpage.find_all('div', attrs={'class' : 'container'})
mmw_calendar_urls = mmw_calendar[0].find_all('a')

mmw_url_list = []
for url in mmw_calendar_urls:
    str_url = str(url)
    mmw_url_list.append(re.search('="(.*)">', str_url).group(1))
mmw_url_list = mmw_url_list[::3]
# print(mmw_url_list)


i = 0
mmw_prices_list = []
for mmw_url in mmw_url_list:
    mmw_dict = {}
    price2 = "Event Sold Out."
    mmw_ticket_response = requests.get(mmw_url, headers=headers)
    mmw_ticket_webpage = BS(mmw_ticket_response.content, 'lxml')
    mmw_list_date = mmw_ticket_webpage.find('time',attrs= {'class' : 'date-time'})
    if mmw_list_date != None:
        
#     print(mmw_list_date)
#     mmw_list_date = re.sub(r'[\w{3}\s\d{2},\s\d{4}]',"", str(mmw_list_date))
        mmw_list_date = re.search('\w{3}\s\d{2},\s\d{4}', str(mmw_list_date)).group(0)
#         print(mmw_list_date)
        mmw_list_artist = mmw_ticket_webpage.find('h1',attrs= {'class' : 'event-name headliners'})
        mmw_list_time = mmw_ticket_webpage.find('span',attrs= {'class' : 'start'})
        mmw_list_price = mmw_ticket_webpage.find('span',attrs= {'class' : 'price-range'})
        mmw_dict['event_date'] = mmw_list_date
        mmw_dict['event_name'] = mmw_list_artist.text
        mmw_dict['show_time'] = mmw_list_time.text.strip()
        mmw_dict['price'] = mmw_list_price.text
        mmw_dict['ticket_url'] = mmw_url
        mmw_prices_list.append(mmw_dict)
        i += 1  
        if i == 9:
            break

In [6]:
mmw_df = pd.DataFrame.from_dict(mmw_prices_list)
mmw_df['price'] = mmw_df['price'].str.replace("$","")
mmw_df['Venue'] = 'Marathon Music Works'
mmw_df['show_time'] = mmw_df.show_time.str.replace(" pm","")
mmw_df['event_date']= pd.to_datetime(mmw_df['event_date'])
mmw_df['headliner'] = mmw_df.event_name.str.replace(" -.*","")

mmw_df

<ipython-input-6-9385459c2af9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  mmw_df['price'] = mmw_df['price'].str.replace("$","")
<ipython-input-6-9385459c2af9>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  mmw_df['headliner'] = mmw_df.event_name.str.replace(" -.*","")


,event_date,event_name,show_time,price,ticket_url,Venue,headliner
0,2022-05-04,SNOH AALEGRA - UGH. THESE TEMPORARY HIGHS TOUR,08:00,48.50 - 59.50,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,SNOH AALEGRA
1,2022-05-07,Emo Night Nashville,08:00,25.00 - 50.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Emo Night Nashville
2,2022-05-10,Stick Figure,08:30,39.50 - 79.50,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Stick Figure
3,2022-05-11,Still Woozy,08:00,25.00 - 55.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Still Woozy
4,2022-05-14,Aly & AJ,08:00,35.00 - 199.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Aly & AJ
5,2022-05-19,Alexisonfire (Moved to Brooklyn Bowl),08:00,41.00 - 81.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Alexisonfire (Moved to Brooklyn Bowl)
6,2022-05-22,Sum 41 & Simple Plan: The Blame Canada Tour,07:00,59.50 - 634.75,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Sum 41 & Simple Plan: The Blame Canada Tour
7,2022-05-29,KASH DOLL / L ROY DA BOY / LIGHT SKIN KEISHA,09:00,50.00 - 100.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,KASH DOLL / L ROY DA BOY / LIGHT SKIN KEISHA
8,2022-06-01,Asking Alexandria + Nothing More – USA Tour 2022,06:35,39.50 - 75.00,https://www.marathonmusicworks.com/event-detai...,Marathon Music Works,Asking Alexandria + Nothing More – USA Tour 2022


In [7]:
mmw_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\mmw_df.csv")

# THE BASEMENT

In [8]:
base_URL = 'https://www.thebasementnashville.com/calendar/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
base_response = requests.get(base_URL, headers=headers)
base_webpage = BS(base_response.content, 'lxml')
base_list = base_webpage.find_all("div", attrs= {"class":"tw-info-price-buy-tix"})
base_list

base_url_list = []
for item in base_list:
    href_sec = item.find_all('a')
    if len(href_sec) > 1:
        base_url_list.append(href_sec[-1]['href'])



# base_url_list = []
# for url in base_list:
#     str_url = str(url)
#     base_url_list.append(re.search('href="(.*)">', str_url).group(1))
# base_url_list

i = 0
base_events_list = []
for base_url in base_url_list:
    if base_url != None:
#         print(base_url)
        base_dict = {}
    #     price3 = "Event Sold Out."
        base_ticket_response = requests.get(base_url, headers=headers)
        base_ticket_webpage = BS(base_ticket_response.content, 'lxml')
        base_list_date = base_ticket_webpage.find('span', attrs= {"class":"tw-event-date"})
        base_list_artist = base_ticket_webpage.find('div', attrs= {"class":"tw-name"})
        base_list_time = base_ticket_webpage.find('span', attrs= {"class":"tw-event-time"})
        base_list_price = base_ticket_webpage.find('div', attrs= {"class":"tw-price"})
        
        if base_list_date != None:
            base_dict['event_date'] = base_list_date.text
        else:
            base_dict['event_date'] = "Date Not Found"
            
        if base_list_artist != None:
            base_dict['event_name'] = base_list_artist.text.strip()
        else:
            base_dict['event_name'] = "Artist Not Found"
            
        if base_list_time != None:
            base_dict['show_time'] = base_list_time.text
        else:
            base_dict['show_time'] = "Time Not Found"
        
        if base_list_price != None:
            base_dict['price'] = base_list_price.text.strip()
        else:
            base_dict['price'] = "Price Not Found"
        base_dict['ticket_url'] = base_url
        base_events_list.append(base_dict)
        i += 1  
        if i == 50:
            break


In [9]:
base_df = pd.DataFrame.from_dict(base_events_list)
base_df['Venue'] = 'The Basement'
base_df['price'] = base_df['price'].str.replace("/"," | ")
base_df['price'] = base_df['price'].str.replace("DOS|day of show","Day of Show")
base_df['show_time'] = base_df.show_time.str.replace(" pm","")
base_df['event_date']= pd.to_datetime(base_df['event_date'])
# base_df['price'] = base_df['price'].str.replace("?:𝐴𝐷𝑉","")
base_df

<ipython-input-9-a0986449d491>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  base_df['price'] = base_df['price'].str.replace("DOS|day of show","Day of Show")


,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-04-29,Kind Hearted Strangers,7:00,$10,https://www.thebasementnashville.com/event/119...,The Basement
1,2022-04-29,SOLD OUT! Los Lobos w/ Ian Ferguson,8:00,$45 ADV | $50 Day of Show,https://www.thebasementnashville.com/event/116...,The Basement
2,2022-04-29,Vandoliers & Sam Morrow,9:00,$12 adv | | $15 Day of Show,https://www.thebasementnashville.com/event/118...,The Basement
3,2022-04-30,Ned Henry w/ The Dambrino Sisters,7:00,$10,https://www.thebasementnashville.com/event/119...,The Basement
4,2022-04-30,The Chats w/ Mean Jeans and THICK,8:00,$20 ADV | $25 Day of Show,https://www.thebasementnashville.com/event/115...,The Basement
5,2022-04-30,Nashville Jamboree Benefit Concert - w/ Stoop ...,9:00,$20,https://www.thebasementnashville.com/event/119...,The Basement
6,2022-05-01,"Nashville Tour Stop: Feat. Dorian Lackey, Just...",7:00,$7 adv | | $10 Day of Show,https://www.thebasementnashville.com/event/119...,The Basement
7,2022-05-01,Gang of Youths - Angel In Realtime Tour w/ Cas...,8:00,$25.00,https://www.thebasementnashville.com/event/115...,The Basement
8,2022-05-02,Brent Anderson w/ Claire Ernst,7:00,$7.00,https://www.thebasementnashville.com/event/120...,The Basement
9,2022-05-02,Sheer Mag & Angel Du$t w/ Snooper,8:00,$17 ADV | $20 Day of Show,https://www.thebasementnashville.com/event/118...,The Basement


In [10]:
base_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\base_df.csv")

# THE BASEMENT EAST

In [11]:
beast_URL = 'https://www.thebasementnashville.com/basement-east-calendar/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
beast_response = requests.get(beast_URL, headers=headers)
beast_webpage = BS(beast_response.content, 'lxml')


beast_list = beast_webpage.find_all("div", attrs= {"class":"tw-info-price-buy-tix"})
beast_url_list = []
for item in beast_list:
    href_sec = item.find_all('a')
    if len(href_sec) > 1:
        beast_url_list.append(href_sec [-1]['href'])
    
i = 0
beast_events_list = []
for beast_url in beast_url_list:
    if beast_url != None:
#         print(base_url)
        beast_dict = {}
    #     price3 = "Event Sold Out."
        beast_ticket_response = requests.get(beast_url, headers=headers)
        beast_ticket_webpage = BS(beast_ticket_response.content, 'lxml')
        beast_list_date = beast_ticket_webpage.find('span', attrs= {"class":"tw-event-date"})
        beast_list_artist = beast_ticket_webpage.find('div', attrs= {"class":"tw-name"})
        beast_list_time = beast_ticket_webpage.find('span', attrs= {"class":"tw-event-time"})
        beast_list_price = beast_ticket_webpage.find('div', attrs= {"class":"tw-price"})
            
        if beast_list_date != None:
            beast_dict['event_date'] = beast_list_date.text
        else:
            beast_dict['event_date'] = beast_list_date2
            
        if beast_list_artist != None:
            beast_dict['event_name'] = beast_list_artist.text.strip()
        else:
            beast_dict['event_name'] = "Artist Not Found"
            
        if beast_list_time != None:
            beast_dict['show_time'] = beast_list_time.text
        else:
            beast_dict['show_time'] = "Time Not Found"
        
        if beast_list_price != None:
            beast_dict['price'] = beast_list_price.text.strip()
        else:
            beast_dict['price'] = "Price Not Found"        
        beast_dict['ticket_url'] = beast_url
        beast_events_list.append(beast_dict)
        i += 1  
        if i == 32:
            break

In [12]:
beast_df = pd.DataFrame.from_dict(beast_events_list)
beast_df['price'] = beast_df['price'].str.replace("$","")
beast_df['price'] = beast_df['price'].str.replace("/"," | ")
beast_df['price'] = beast_df['price'].str.replace("DOS|day of show","Day of Show")
beast_df['show_time'] = beast_df.show_time.str.replace(" pm","")
beast_df['event_date']= pd.to_datetime(beast_df['event_date'])
beast_df['Venue'] = 'The Basement East'

beast_df

<ipython-input-12-5a5113210bc9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  beast_df['price'] = beast_df['price'].str.replace("$","")
<ipython-input-12-5a5113210bc9>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  beast_df['price'] = beast_df['price'].str.replace("DOS|day of show","Day of Show")


,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-04-29,SOLD OUT! Los Lobos w/ Ian Ferguson,8:00,45 ADV | 50 Day of Show,https://www.thebasementnashville.com/event/116...,The Basement East
1,2022-04-30,The Chats w/ Mean Jeans and THICK,8:00,20 ADV | 25 Day of Show,https://www.thebasementnashville.com/event/115...,The Basement East
2,2022-05-01,Gang of Youths - Angel In Realtime Tour w/ Cas...,8:00,25.00,https://www.thebasementnashville.com/event/115...,The Basement East
3,2022-05-02,Sheer Mag & Angel Du$t w/ Snooper,8:00,17 ADV | 20 Day of Show,https://www.thebasementnashville.com/event/118...,The Basement East
4,2022-05-03,Clinton Kane w/ Lexi Jayde - MAYBE SOMEDAY IT'...,8:00,25-125,https://www.thebasementnashville.com/event/115...,The Basement East
5,2022-05-04,Built to Spill w/ Itchy Kitty and Prism Bitch,8:00,27 ADV | 30 Day of Show,https://www.thebasementnashville.com/event/116...,The Basement East
6,2022-05-05,The Menzingers w/ Oso Oso and Sincere Engineer,8:00,23 ADV | 25 Day of Show,https://www.thebasementnashville.com/event/115...,The Basement East
7,2022-05-06,SOLD OUT! Teddy Swims - Tough Love World Tour ...,8:00,23.50-110,https://www.thebasementnashville.com/event/113...,The Basement East
8,2022-05-07,The Taylor Party: TAYLOR SWIFT NIGHT,9:00,12-25,https://www.thebasementnashville.com/event/118...,The Basement East
9,2022-05-08,The Band of Heathens w/ Chicago Farmer,8:00,17 ADV | 20 Day of Show,https://www.thebasementnashville.com/event/116...,The Basement East


In [13]:
beast_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\beast_df.csv")

# Mercy Lounge

In [17]:
mrcy_URL = 'https://www.mercylounge.com/calendar/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
mrcy_response = requests.get(mrcy_URL, headers=headers)
mrcy_webpage = BS(mrcy_response.content, 'lxml') 
a_list = mrcy_webpage.find_all("a")
a_list

mrcy_url_list = []
for a in a_list[14:]:
    mrcy_url_list.append(a['href'])

mrcy_url_list = mrcy_url_list[::3]
mrcy_urls = []
for url in mrcy_url_list:
    url = url.replace("eventbrite","mercylounge")
    url = url.replace("tickets-","")
    url = url.replace("#tickets","/")
    mrcy_urls.append(url)

mrcy_urls
i = 0
mrcy_events_list = []
for mrcy_url in mrcy_urls:
    if mrcy_url != None:
#         print(mrcy_url)
        mrcy_dict = {}
        price3 = "Price Not Listed"
        mrcy_ticket_response = requests.get(mrcy_url, headers=headers)
        mrcy_ticket_webpage = BS(mrcy_ticket_response.content, 'lxml')
        mrcy_list_date = mrcy_ticket_webpage.find('span', attrs= {"class":"dates"})
        mrcy_list_artist = mrcy_ticket_webpage.find('h1', attrs= {"class":"event-name headliners"})
        mrcy_list_time = mrcy_ticket_webpage.find('span', attrs= {"class":"start"})
        mrcy_list_price = mrcy_ticket_webpage.find('span', attrs= {"class":"price-range"})
        mrcy_dict['event_date'] = mrcy_list_date.text
        mrcy_dict['event_name'] = mrcy_list_artist.text
        mrcy_dict['show_time'] = mrcy_list_time.text
        if mrcy_list_price != None:
            mrcy_dict['price'] = mrcy_list_price.text
        else:
            mrcy_dict['price'] = price3  
        mrcy_dict['ticket_url'] = mrcy_url
        mrcy_events_list.append(mrcy_dict)
        i += 1  
        if i == 26:
            break

In [16]:
mrcy_df = pd.DataFrame.from_dict(mrcy_events_list)
# beast_df['price'] = beast_df['price'].str.replace("$","\$")
mrcy_df

mrcy_df['event_date'] = mrcy_df.event_date.str.replace(".*(·)","")
mrcy_df['show_time'] = mrcy_df.show_time.str.replace("Show: ","")
mrcy_df['show_time'] = mrcy_df.show_time.str.replace(" PM",":00")
mrcy_df['event_date']= pd.to_datetime(mrcy_df['event_date'])
mrcy_df['Venue'] = 'Mercy Lounge'
mrcy_df

<ipython-input-16-3c4108c44cea>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  mrcy_df['event_date'] = mrcy_df.event_date.str.replace(".*(·)","")


,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-04-29,Tristen w/ The Prescriptions & Abby Johnson,8:00,$15.00,https://www.mercylounge.com/e/tristen-w-the-pr...,Mercy Lounge
1,2022-04-29,Dan Andriano & The Bygones w/ Emily Wolfe,8:30:00,$19.00 - $22.00,https://www.mercylounge.com/e/dan-andriano-the...,Mercy Lounge
2,2022-04-30,The Story So Far,7:30:00,$44.50,https://www.mercylounge.com/e/the-story-so-far...,Mercy Lounge
3,2022-04-30,Destroyer w/ Rosali,8:00,$20.00 - $25.00,https://www.mercylounge.com/e/destroyer-w-rosa...,Mercy Lounge
4,2022-05-01,Kevin Devine,7:30:00,$18.50 - $23.00,https://www.mercylounge.com/e/kevin-devine-229...,Mercy Lounge
5,2022-05-02,Allie X,8:00,$20.00 - $25.00,https://www.mercylounge.com/e/allie-x-86161570...,Mercy Lounge
6,2022-05-02,Bonnie ‘Prince’ Billy & Matt Sweeney “Superwol...,8:00,$30.00,https://www.mercylounge.com/e/bonnie-prince-bi...,Mercy Lounge
7,2022-05-05,James Arthur: It’ll All Make Sense Tour,8:00,$35.00 - $119.00,https://www.mercylounge.com/e/james-arthur-itl...,Mercy Lounge
8,2022-05-05,Gavin DeGraw: Full Circle Tour - Night 1,8:00,Price Not Listed,https://www.mercylounge.com/e/gavin-degraw-ful...,Mercy Lounge
9,2022-05-06,Swift and Sour: A Taylor & Olivia Dance Party,8:00,$25.00,https://www.mercylounge.com/e/swift-and-sour-a...,Mercy Lounge


In [18]:
mrcy_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\mrcy_df.csv")

# Ryman 

In [19]:
rymn_URL = 'https://ryman.com/events/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
rymn_response = requests.get(rymn_URL, headers=headers)
rymn_webpage = BS(rymn_response.content, 'lxml') 

rymn_narrow = rymn_webpage.find_all("a")
rymn_narrow
rymn_url_list = []
for a in rymn_narrow[48:]:
    if "ryman.com/event" in a['href']:
        rymn_url_list.append(a['href'])
    
rymn_url_list
rymn_url_list = list(pd.Series(rymn_url_list).unique())

i = 0
rymn_events_list = []
for rymn_url in rymn_url_list:
    if rymn_url != None:
#         print(mrcy_url)
        rymn_dict = {}
        price3 = "Price Not Listed"
        rymn_ticket_response = requests.get(rymn_url, headers=headers)
        rymn_ticket_webpage = BS(rymn_ticket_response.content, 'lxml')
        rymn_list_date = rymn_ticket_webpage.find('div', attrs= {"class":"start-time"})
        rymn_list_date1 = re.search('(\w*\s\d*,\s\d{4})', str(rymn_list_date)).group(0)
        rymn_list_artist = rymn_ticket_webpage.find('h1', attrs= {"class":"concert"})
        rymn_list_time = re.search('\s(\d.\d+\s\w\w\s)CDT', str(rymn_list_date)).group(0)
        rymn_list_price = rymn_ticket_webpage.find('p', attrs= {"class":"theprices"})
        rymn_dict['event_date'] = rymn_list_date1
        rymn_dict['event_name'] = rymn_list_artist.text
        rymn_dict['show_time'] = rymn_list_time.replace("CDT","").strip()
        if rymn_list_price != None:
            rymn_dict['price'] = rymn_list_price.text.strip()
        else:
            rymn_dict['price'] = price3  
        rymn_dict['ticket_url'] = rymn_url
        rymn_events_list.append(rymn_dict)
        i += 1  
        if i == 14:
            break

In [20]:
rymn_df = pd.DataFrame.from_dict(rymn_events_list)
rymn_df['Venue'] = 'Ryman Auditorium'
rymn_df['event_date']= pd.to_datetime(rymn_df['event_date'])
rymn_df['show_time'] = rymn_df.show_time.str.replace(" PM","")

rymn_df

,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-04-29,Walker Hayes,8:00,$50 / $30 / $25,https://ryman.com/event/walker-hayes/,Ryman Auditorium
1,2022-04-30,Trampled By Turtles,8:00,$50 / $39.50,https://ryman.com/event/trampled-by-turtles-04...,Ryman Auditorium
2,2022-05-01,Kurt Vile and The Violators,7:30,$48 / $38 / $33,https://ryman.com/event/kurt-vile/,Ryman Auditorium
3,2022-05-03,Van Morrison,7:30,$149.50 / $129.50 / $99.50 / $59.50,https://ryman.com/event/van-morrison-050322/,Ryman Auditorium
4,2022-05-04,Van Morrison,7:30,$149.50 / $129.50 / $99.50 / $59.50,https://ryman.com/event/van-morrison-050422/,Ryman Auditorium
5,2022-05-05,Melissa Etheridge,7:30,$75 / $59.50 / $49.50,https://ryman.com/event/melissa-etheridge-050522/,Ryman Auditorium
6,2022-05-06,Spring Plaza Party,6:00,Free and Open to the Public,https://ryman.com/event/spring-plaza-party-050...,Ryman Auditorium
7,2022-05-06,Billy Strings,8:30,$74.50 / $59.50 / $49.50 / $39.50,https://ryman.com/event/billy-strings-050622/,Ryman Auditorium
8,2022-05-07,Billy Strings,8:30,$74.50 / $59.50 / $49.50 / $39.50,https://ryman.com/event/billy-strings-050722/,Ryman Auditorium
9,2022-05-08,Billy Strings,8:30,$74.50 / $59.50 / $49.50 / $39.50,https://ryman.com/event/billy-strings-050822/,Ryman Auditorium


In [21]:
rymn_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\rymn_df.csv")

# The 5 Spot

In [22]:
spot_URL = 'https://the5spot.club/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
spot_response = requests.get(spot_URL, headers=headers)
spot_webpage = BS(spot_response.content, 'lxml') 

spot_narrow = spot_webpage.find("div", attrs= {"class":"tribe-events-calendar-month__week"})
# for div in spot_narrow.find_all("div", {'class':'tribe-events-calendar-month__day tribe-events-calendar-month__day--past'}): 
#     div.decompose()

spot_narrow = spot_narrow.find_all('a')
    

spot_url_list = []
for a in spot_narrow:
    if ".club/event/" in a['href'] and '/all/' not in a['href']:
        if a['href'] not in spot_url_list:
            spot_url_list.append(a['href'])
        
spot_url_list

i = 0
spot_events_list = []
for spot_url in spot_url_list:
    if spot_url != None:
#         print(base_url)
        spot_dict = {}
    #     price3 = "Event Sold Out."
        spot_ticket_response = requests.get(spot_url, headers=headers)
        spot_ticket_webpage = BS(spot_ticket_response.content, 'lxml')
        spot_list_date = spot_ticket_webpage.find('span', attrs= {"class":"tribe-event-date-start"})
        spot_list_date1 = re.search('>(.*)@', str(spot_list_date)).group(1).strip()
        spot_list_artist = spot_ticket_webpage.find('h1', attrs= {"class":"tribe-events-single-event-title"})
        spot_list_time = re.search('@\s(.*)<', str(spot_list_date)).group(1).strip()
        spot_list_price = spot_ticket_webpage.find('span', attrs= {"class":"tribe-events-cost"})
            
        if spot_list_date != None:
            spot_dict['event_date'] = spot_list_date1
        else:
            spot_dict['event_date'] = "No Date Listed"
            
        if spot_list_artist != None:
            spot_dict['event_name'] = spot_list_artist.text.strip()
        else:
            spot_dict['event_name'] = "Artist Not Found"
            
        if spot_list_time != None:
            spot_dict['show_time'] = spot_list_time
        else:
            spot_dict['show_time'] = "Time Not Found"
        
        if spot_list_price != None:
            spot_dict['price'] = spot_list_price.text
        else:
            spot_dict['price'] = "$10"
        spot_dict['ticket_url'] = spot_url
        spot_events_list.append(spot_dict)
        i += 1  
        if i == 40:
            break
spot_events_list

[{'event_date': 'March 27',
  'event_name': '“Sunday Night Soul”',
  'show_time': '6:00 pm',
  'price': '$10',
  'ticket_url': 'https://the5spot.club/event/sunday-night-soul-10/'},
 {'event_date': 'March 28',
  'event_name': 'Blue Monday with The Tiger Beats',
  'show_time': '6:00 pm',
  'price': '$5',
  'ticket_url': 'https://the5spot.club/event/blue-monday-with-the-tiger-beats-109/2022-03-28/'},
 {'event_date': 'March 28',
  'event_name': 'MOTOWN MONDAY dance party',
  'show_time': '9:00 pm',
  'price': '$5',
  'ticket_url': 'https://the5spot.club/event/motown-monday-dance-party/2022-03-28/'},
 {'event_date': 'March 29',
  'event_name': 'DeadBeats Tie Dye Jam $5',
  'show_time': '6:00 pm',
  'price': '$10',
  'ticket_url': 'https://the5spot.club/event/deadbeats-tie-dye-jam/'},
 {'event_date': 'March 29',
  'event_name': 'Two Dollar Tues with Derek Hoke',
  'show_time': '8:00 pm',
  'price': '$2',
  'ticket_url': 'https://the5spot.club/event/two-dollar-tues-with-derek-hoke/2022-03-29/

In [23]:
spot_df = pd.DataFrame.from_dict(spot_events_list)
spot_df['Venue'] = 'The 5 Spot'
spot_df['show_time'] = spot_df.show_time.str.replace(" pm","")
spot_df['event_date'] = spot_df['event_date'].astype(str) + ', 2022'
spot_df['event_date']= pd.to_datetime(spot_df['event_date'])
spot_df

,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-03-27,“Sunday Night Soul”,6:00,$10,https://the5spot.club/event/sunday-night-soul-10/,The 5 Spot
1,2022-03-28,Blue Monday with The Tiger Beats,6:00,$5,https://the5spot.club/event/blue-monday-with-t...,The 5 Spot
2,2022-03-28,MOTOWN MONDAY dance party,9:00,$5,https://the5spot.club/event/motown-monday-danc...,The 5 Spot
3,2022-03-29,DeadBeats Tie Dye Jam $5,6:00,$10,https://the5spot.club/event/deadbeats-tie-dye-...,The 5 Spot
4,2022-03-29,Two Dollar Tues with Derek Hoke,8:00,$2,https://the5spot.club/event/two-dollar-tues-wi...,The 5 Spot
5,2022-03-30,"Mike Younger, Charlie Treat",6:00,$10,https://the5spot.club/event/mike-younger-charl...,The 5 Spot
6,2022-03-30,lespecial // Spirits Republic,9:00,$10,https://the5spot.club/event/lespecial/,The 5 Spot
7,2022-03-31,"Caroline Kid, Amanda Fama",6:00,$10,https://the5spot.club/event/caroline-kid-amand...,The 5 Spot
8,2022-03-31,"Eyelids (members of Guided By Voices, December...",9:00,$10,https://the5spot.club/event/eyelids-members-of...,The 5 Spot
9,2022-04-01,Tim Carroll Rock’n’Roll Happy Hour !!!,6:00,$10,https://the5spot.club/event/tim-carroll-rocknr...,The 5 Spot


In [24]:
spot_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\spot_df.csv")

# The End

In [25]:
end_URL = 'https://endnashville.com/events/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
end_response = requests.get(end_URL, headers=headers)
end_webpage = BS(end_response.content, 'lxml') 

end_narrow = end_webpage.find("div", attrs={"class":"tribe-events-loop vcalendar"})
end_urls = end_narrow.find_all("a", attrs={"class":"url"})

end_url_list = []
for url in end_urls:
    if url['href'] not in end_url_list:
        end_url_list.append(url['href'])
end_url_list

i = 0
end_events_list = []
for end_url in end_url_list:
    if end_url != None:
        end_dict = {}
        end_ticket_response = requests.get(end_url, headers=headers)
        end_ticket_webpage = BS(end_ticket_response.content, 'lxml')
        end_list_date = end_ticket_webpage.find('p', attrs= {"class":"rhino-event-date"})
        end_list_artist = end_ticket_webpage.find('a', attrs= {"class":"url"})
        end_list_time = end_ticket_webpage.find('p', attrs= {"class":"rhino-event-time"})
        end_list_price = end_ticket_webpage.find('p', attrs= {"class":"rhino-event-price"})
            
        if end_list_date != None:
            end_dict['event_date'] = end_list_date.text
        else:
            end_dict['event_date'] = "No Date Listed"
            
        if end_list_artist != None:
            end_dict['event_name'] = end_list_artist.text.strip()
        else:
            end_dict['event_name'] = "Artist Not Found"
            
        if end_list_time != None:
            end_dict['show_time'] = end_list_time.text
        else:
            end_dict['show_time'] = "Time Not Found"
        
        if end_list_price != None:
            end_dict['price'] = end_list_price.text.strip()
        else:
            end_dict['price'] = "Price Not Listed"
        end_dict['ticket_url'] = end_url
        end_events_list.append(end_dict)
        i += 1  
        if i == 21:
            break

In [26]:
end_df = pd.DataFrame.from_dict(end_events_list)
end_df['event_date'] = end_df.event_date.str.replace("(.*,)","")
end_df['show_time'] = end_df.show_time.str.replace("(Show :)","")
end_df['show_time'] = end_df.show_time.str.replace(" pm","")
end_df['event_date'] = end_df['event_date'].astype(str) + ', 2022'
end_df['event_date']= pd.to_datetime(end_df['event_date'])
end_df['Venue'] = 'The End'
end_df

<ipython-input-26-0f1a251dd8dd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  end_df['event_date'] = end_df.event_date.str.replace("(.*,)","")
<ipython-input-26-0f1a251dd8dd>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  end_df['show_time'] = end_df.show_time.str.replace("(Show :)","")


,event_date,event_name,show_time,price,ticket_url,Venue
0,2022-04-29,Live Emo Cover Night: Feat. Ellie Stone w/The ...,8:00,$10,https://endnashville.com/event/live-emo-cover-...,The End
1,2022-04-30,*Cancelled* Mac Lethal w/Feral The Earthworm –...,8:00,Price Not Listed,https://endnashville.com/event/cancelled-mac-l...,The End
2,2022-04-30,Garden Of Eden w/Red Ransom & Hippies & Cowboy...,8:00,$8,https://endnashville.com/event/garden-of-eden-...,The End
3,2022-05-01,Exociety presents: The Out Of The Woods Tour f...,8:00,$15,https://endnashville.com/event/exociety-presen...,The End
4,2022-05-04,"Sugar In The Gas Tank w/ Platinum Moon, Keep T...",8:00,$8,https://endnashville.com/event/sugar-in-the-ga...,The End
5,2022-05-05,Legacy Concerts Presents: Essenger – 18+,8:00,$13,https://endnashville.com/event/legacy-concerts...,The End
6,2022-05-06,Cam Cole w/ Volcano Girls – 18+,8:00,$10,https://endnashville.com/event/cam-cole-w-volc...,The End
7,2022-05-07,"Lydia’s Castle EP Release w/ True Villains, Vo...",8:00,$10,https://endnashville.com/event/lydias-castle-e...,The End
8,2022-05-11,SOLD OUT: Legacy Concerts Presents: Peter McPo...,8:00,Price Not Listed,https://endnashville.com/event/sold-out-legacy...,The End
9,2022-05-12,Felix Tandem w/Gentry Blue & Throw Pillows – 18+,8:00,$8,https://endnashville.com/event/felix-tandem-w-...,The End


In [27]:
end_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\end_df.csv", index=False)

# Nashville Symphony

In [ ]:
# symp_URL = 'https://www.nashvillesymphony.org/tickets'
# headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
# symp_response = requests.get(symp_URL, headers=headers)
# symp_webpage = BS(symp_response.content, 'lxml') 

# symp_narrow = symp_webpage.find('div', attrs={'class':'events'})
# symp_url_list = []
# for link in symp_narrow.find_all('a'):
#     if link.get('href') != None:
#         if 'nashvillesymphony' in link.get('href'):
#             symp_url_list.append(link.get('href'))
# symp_url_list

# for url in symp_url_list:
#     symp_ticket_response = requests.get(url)
#     symp_ticket_webpage = BS(symp_ticket_response.content, 'lxml')
#     print(symp_ticket_webpage)
# i = 0
# symp_events_list = []
# for symp_url in symp_web[]:
#     if symp_url != None:
#         symp_dict = {}
#         symp_ticket_response = requests.get(symp_url, headers=headers)
#         symp_ticket_webpage = BS(symp_ticket_response.content, 'lxml')
#         symp_list_date = symp_ticket_webpage.find('p', attrs= {"class":"tn-event-detail__display-time"})
#         symp_list_artist = symp_ticket_webpage.find('p', attrs= {"class":"tn-event-detail__title"})
#         symp_list_time = symp_ticket_webpage.find('p', attrs= {"class":"rhino-event-time"})
#         symp_list_price = "Click For Ticket Prices."
            
#         if symp_list_date != None:
#             symp_dict['event_date'] = symp_list_date.text
#         else:
#             symp_dict['event_date'] = "No Date Listed"
            
#         if symp_list_artist != None:
#             symp_dict['event_name'] = symp_list_artist.text.strip()
#         else:
#             symp_dict['event_name'] = "Artist Not Found"
            
#         if symp_list_time != None:
#             symp_dict['show_time'] = symp_list_time.text
#         else:
#             symp_dict['show_time'] = "Time Not Found"
        
#         if symp_list_price != None:
#             symp_dict['price'] = symp_list_price

#         symp_dict['ticket_url'] = symp_url
#         symp_events_list.append(symp_dict)
#         i += 1  
#         if i == 21:
#             break

# Plaza Mariachi

In [ ]:
plza_URL = 'https://plazamariachi.com/events/month/2022-05/'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'} 
plza_response = requests.get(plza_URL, headers=headers)
plza_webpage = BS(plza_response.content, 'lxml') 

print(plza_webpage)

# plza_webpage.find_all('h3', attrs= {'class':'tribe-events-calendar-month-mobile-events__mobile-event-title tribe-common-h7'})
# plza_webpage.find_all('span', attrs= {'class':'tribe-event-date-start'})

# # plza_url_list = []
# # for link in plza_webpage.find_all('a')[51:220:]:
# #     if link.get('href') != None:
# #         if '/all/' not in link.get('href'):
# #             if 'plazamariachi.com/event' in link.get('href'):
# #                 if '2022-05' not in link.get('href'):
# #                     if link.get('href') not in plza_url_list:
# #                         plza_url_list.append(link.get('href'))
# # plza_url_list

# plza_artist_list = []
# plza_dt_list = []
# plza_time_list = []
# plza_date_list = []
# plza_tickets_list = []

# plza_artist = plza_webpage.find_all('h3', attrs= {'class':'tribe-events-calendar-month__calendar-event-title tribe-common-h8 tribe-common-h--alt'})
# for artist in plza_artist:
#     plza_artist_list.append(artist.text.strip())
# plza_artist_list

# plza_dt = plza_webpage.find_all('span', attrs= {'class':'tribe-event-date-start'})
# for dt in plza_dt[37::]:
#     plza_dt_list.append(dt.text.strip())
# plza_dt_list


# for dt in plza_dt_list:
#     split_string = dt.partition(' @ ')
#     plza_date_list.append(split_string[0])
#     plza_time_list.append(split_string[2])
    
# plza_time_list
# plza_df = pd.DataFrame(plza_dict)
# plza_df = pd.DataFrame()

# plza_dict = {'event_date':plza_date_list,'event_name':plza_artist_list, 'show_time':plza_time_list, 'price':'Free', 'ticket_url':'No URL found', 'Venue':'Plaza Mariachi'}

# plza_df = pd.DataFrame(plza_dict)
# plza_df['event_date']= pd.to_datetime(plza_df['event_date'])
# plza_df = pd.DataFrame(plza_dict)
# plza_df['event_date'] = plza_df['event_date'].astype(str) + ', 2022'
# plza_df['event_date']= pd.to_datetime(plza_df['event_date'])

# plza_df

In [ ]:
whole_thing = pd.read_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\venues_df.csv")

In [ ]:
# whole_thing['event_date']= pd.to_datetime(whole_thing['event_date'])
# whole_thing.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\venues_df.csv", index=False)

In [ ]:
venues = [mmw_df, base_df, beast_df, mrcy_df, rymn_df, spot_df, end_df, plza_df]

venues_df = pd.concat(venues)

In [ ]:
# venues_df.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\WhatchaWannaDo\Data\venues_df.csv", index=False)

In [ ]:
whole_thing['event_date']= pd.to_datetime(whole_thing['event_date'])
whole_thing.to_csv(r"C:\Users\jal32\Documents\NSS\Capstones\venues_df.csv", index=False)

In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# import pandas as pd
# import sys
# import time 
# import requests

In [ ]:
# client_id = '9e90f26f3db84262b5867774570fea6d'
# client_secret = 'ea2b3450773b487498cd42f8408b0f83'

# client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# sp.search(q="garden of eden", type="artist", limit=10)

In [ ]:
# whole_thing.to_csv('../')